## Friday 9

files needed = sleep75.dta, wage1.dta

This week we are working on

1. Exporting figures and tables
2. OLS


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Linear regression

This notebook introduces us to the statsmodels package [(docs)](https://devdocs.io/statsmodels/), which provides functions for formulating and estimating statistical models. This notebook will not address the models, per se, but will focus on how to put econometrics to work in python.

Many of you have used STATA before. STATA is a great package for econometrics. Python can do most of what STATA can do, but STATA will have more specialized routines available. As python's popularity grows the kinds of models you can estimate in it grows, too.    

If STATA is your thing, this [page](http://rlhick.people.wm.edu/posts/comparing-stata-and-ipython-commands-for-ols-models.html) on Rob Hicks' website is a nice STATA-to-python concordance.

In [ ]:
import pandas as pd                    # for data handling
import numpy as np                     # for numerical methods and data structures
import matplotlib.pyplot as plt        # for plotting


# The new package
import statsmodels.formula.api as smf  # provides a way to directly spec models from formulas

### Reading Stata data files

Jeff Wooldridge's econometrics textbooks are academic staples. Our plan today is to work through some of the problems in the Wooldridge textbook as a way to introduce regression in python. 

On the plus side, the data that correspond to the Wooldridge problems are available to download and they are **ALREADY CLEANED.** \[I contemplated adding some junk to the files to make it more interesting...\]

On the minus side, the files are in STATA's .dta format. 

Lucky for us, pandas has a method that [reads stata files](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_stata.html). It also has methods for SQL, SAS, JSON,...

In [ ]:
# Use pandas read_stata method to get the stata formatted data file into a DataFrame.
sleep = pd.read_stata('sleep75.dta')

# Take a look...so clean!
sleep.head()

In [ ]:
# Another method for checking out a DataFrame
sleep.info()

### Specifying and estimating models with the formula.api

Consider  the regression model

$$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + \varepsilon,$$

where $y$,  $x_1$, and $x_2$ are variables, $\beta_0, \beta_1$ and $\beta_2$ are the parameters to estimate, and $\epsilon$ is the error term. 


The `statsmodels` package provides us with a formulaic syntax for defining models that uses strings. The basic syntax is 
```
y ~ x1 + x2
```
which describes the model above. Notice that I did not specify the constant. Statsmodels takes care of that automatically.


The work flow is:
1. Specify the regression: sort out the dependent and independent variables
2. Create the model with statsmodel
3. Fit the model and obtain results

To do this, we use the `statsmodels.formula.api` methods, which we imported as `smf`. 

## 1. Specify the regression

How do hours of sleep vary with working? Do we trade off sleep for work? We control for education and age.

$$ sleep = \beta_0 + \beta_1 totwrk + \beta_2 educ + \beta_3 age + \epsilon. $$

\[This is in problem 3, chapter 3 in Wooldridge.\]

## 2. Create the model

Using the statsmodel syntax, we have 

```python
sleep ~ totwrk + educ + age
```

Remember, the constant is automatically added. 

We use the `.ols()` method of statsmodels. This is the *ordinary least squares* model. 

In [ ]:
sleep_model = smf.ols('sleep ~ totwrk + educ + age', data=sleep)
type(sleep_model)

The `model.OLS` object contains information about the regression model. Things like:

* sleep_model.exog_names
* sleep_model.endog_names
* sleep_model.nobs

Check the documentation or try `sleep_model.` and then `TAB`. 

In [ ]:
# Names of the exogenous (the right-hand side) variables in the model. 
sleep_model.exog_names

## 3. Estimate the model

Step \#2 set up the model, but did not estimate the coefficients. To estimate the model, we use the `.fit()` method of the OLS model object. 

In [ ]:
results = sleep_model.fit()
type(results)

Another object! This time, a `RegressionResultsWrapper`. This object hold all the, well, results. Try `results.` and `TAB` again to see what lives in there.

In [ ]:
print(results.summary())

Side note: Remember that in regression, we're essentially just solving a system of equations a la $Ax = B$. So, I'll also show that our results can be replicated using NumPy and some linear algebra.

If you imagine $\beta$ to be a vector $(\beta_0, \beta_1, \beta_2, \beta_3)^T$, then our data just becomes a matrix:

$$X = \begin{bmatrix} 1 & totwrk_0 & educ_0 & age_0 \\ \vdots & \vdots & \vdots & \vdots \\ 1 & totwrk_n & educ_n & age_n \end{bmatrix}$$

So then our equation just looks like this:

\begin{align}
y &= X\beta \\
X'y &= X'X\beta \\
(X'X)^{-1} X'y &= (X'X)^{-1} X'X \beta \\
(X'X)^{-1} X'y &= \beta
\end{align}

In [ ]:
# this just prevents the output from being scientific notation
np.set_printoptions(suppress=True)

In [ ]:
sleep.shape

In [ ]:
# create the column vector (our dependent variable)
sleepy = sleep['sleep'].to_numpy()

sleepmat = np.ones((sleep.shape[0],4)) # create a matrix of ones to be our constant
sleepmat[:,1:] = sleep[['totwrk','educ','age']].to_numpy() # replace the second thru fourth columns with data
sleepmat

In [ ]:
# the @ symbol is NumPy shorthand for matrix multiplication
# Here, we're solving Ax = B as above
sleep_b = np.linalg.inv(sleepmat.T @ sleepmat) @ sleepmat.T @ sleepy
sleep_b

In [ ]:
# We can also use the lstsq command in NumPy to do this a little quicker
sleep_b_solve = np.linalg.lstsq(sleepmat, sleepy)
sleep_b_solve # lstsq returns (1) coefficients, (2) SSR, (3) matrix rank, and (4) singular values (from SVD of A)

In [ ]:
print(results.summary())

So, we can see by comparing the three that all of them give the same results. Feel free to use whichever is most convenient to you. Back to the original work...

The more you work, the less you sleep. I feel better already.

We can retrieve individual results from the RegressionResultsWrapper object. 

In [ ]:
print('The parameters are:\n', results.params, '\n')
print('The confidence intervals are:\n', results.conf_int(), '\n')
print('The r-sqared is:', results.rsquared)

### 1-2-3: Doing it all at once

We can chain together the three steps into one line of code. 

In [ ]:
res = smf.ols('sleep ~ totwrk + educ + age', data=sleep).fit()
print(res.summary())

## Transforming data

#### logs 

Logs get used a lot in econometrics. Regressing the log of a variable on the log of another variable means that the estimated coefficient is interpreted as an elasticity. 

We use the `np.log( )` method directly in the regression specification syntax. Note that we loaded the numpy package above as `np`. Let's modify our model and use the logarithm of age

$$ sleep = \beta_0 + \beta_1 totwrk + \beta_2 educ + \beta_3 \log(age)  + \epsilon. $$

The notation is 

```python
sleep ~ totwrk + educ + np.log(age)
```


In [ ]:
res = smf.ols('sleep ~ totwrk + educ + np.log(age)', data=sleep).fit()
print(res.summary())

## Practice

Take a few minutes and try the following. Feel free to chat with those around you if you get stuck. I am here, too. 

Wooldridge problem C2 in chapter 6. 

1. Load wage1.dta
2. Scatter plot log(wage) against educ. There is a log-wage variable in the data already, but let's practice transforming data. Create your own log-wage variable. \[You might use `.apply()` or `.map()`\]

3. Estimate 
$$ \log(wage) = \beta_0 + \beta_1 educ + \epsilon$$

**Name your results object** `prac_results`. 

In [ ]:
# 1 and 2



In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

# Code here



In [ ]:
# 3

# Code here

Shifting gears... what if we wanted to put all this into our project writeup at the end of the semester?

## Adding figures to MS Word. 

Part of your final project is an executive summary formatted as a PDF. A natural way to create the PDF is to create an MS Word document and export it as a PDF at the end. 

This means you will need to get your figures from a Jupyter notebook to an MS Word document. There are many ways to do this. The best way to do this is to first save your figure from inside your notebook, and then add it to your Word document. 

**Do not use a screenshot or the snipping tool to take a picture of the figure in your notebook and paste it into your document.** This will result in blurry figures that can be difficult to read. 

**Note from Greg**: I will also create a LaTeX template document for those who would like to create a more professional summary. Check for it at the end of the weekend if it hasn't been posted already.

Let's save our plot of education and log-wages. We will save the figure in two formats. 

The first is PNG, which is a [raster graphics](https://en.wikipedia.org/wiki/Raster_graphics) format. A raster graphic is essentially made up of many tiny dots. 

The second is SVG, which is a [scalable vector](https://en.wikipedia.org/wiki/Scalable_Vector_Graphics) graphics format. The image is essentially a set of equations that describe the shape. 

matplotlib handles both these formats easily.

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

# Code Here

plt.show()

### No title?

Notice that I did not add a title to the figure. This seems like malpractice. 

I did this because I will add the title in MS Word. The title is integrated into the caption and figure number object in Word. 

## Practice: Adding figures to Word

1. Open a new MS Word document. 

2. Add the two figures we created in cell above. To add a figure to Word, use the `Insert` tab at the top of the document and choose `Pictures`.

3. Add a caption. Right click on the figure and choose `Insert caption`. Set the `Label` field to `Figure`. I like my captions above the figure, but you can put them below if you choose. Make your captions consistent in their appearance and location.


### The importance of vector graphics

4. Zoom in on your document. The zoom control is in the lower-right corner. Do you see how blurry the PNG figure has become? The SVG figure scales smoothly with the figure's size. 

Be sure to use SVG files when creating your project documents. 

### Exporting regression tables

This area is less refined than exporting figures, unfortunately. Thanks to [this nice StackExchange post](https://economics.stackexchange.com/questions/11774/outputting-regressions-as-table-in-python-similar-to-outreg-in-stata), I did manage to find one solution for exporting from Python to LaTex. If you intend to use Word for your output, I would recommend using the `to_csv` command and then copy/pasting from there into a Word table as necessary.

In [ ]:
print(res.summary())

beginningtex = """\\documentclass{report}
\\usepackage{booktabs}
\\begin{document}"""
endtex = "\end{document}"

f = open('reg_practice.tex', 'w')
f.write(beginningtex)
f.write(res.summary().as_latex())
f.write(endtex)
f.close()

In [ ]:
f = open('csv_practice.csv', 'w')
f.write(res.summary().as_csv())
f.close()